In [4]:
''' Imports and function to get image pathces into bags with an assigned bag label without augmentation on patches '''
import os
import glob
from PIL import Image
import numpy as np
import torch
from torchvision import datasets, transforms
import random
class PatchMethod(torch.utils.data.Dataset):
    def __init__(self, root = 'USCB/train/*/*/*/*.tif', mode = 'train', transform = None):
        self.root = root
        self.mode = mode
        self.raw_samples = glob.glob(root)
        self.samples = []
        for raw_sample in self.raw_samples:
            self.samples.append((raw_sample, int(raw_sample.split('\\')[-3])))

    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        if self.mode == 'train':
            random.shuffle(self.samples)
            
        image_dir, label = self.samples[index]
        images = glob.glob(image_dir)
   
        
        transformations = transforms.Compose([
            transforms.ToTensor()
        ])
        
        array = []
        
        for i, image_path in enumerate(images):
            image = Image.open(image_path)
            image = np.array(image)
            r, c, _ = image.shape
            for i in range(0,32*24,32):
                for j in range(0,32*28,32):
                    array.append(transformations(image[i:i+32, j:j+32, :]))

                    
                    
        array = tuple(array)
 
        array = torch.stack(array, 0)
        
        return (array, label)

In [5]:
''' Function to get image pathces into bags with an assigned bag label with augmentation on patches '''
class AugPatchMethod(torch.utils.data.Dataset):
    def __init__(self, root = 'USCB/train/*/*/*/*.tif', mode = 'train', transform = None):
        self.root = root
        self.mode = mode
        self.raw_samples = glob.glob(root)
        self.samples = []
        for raw_sample in self.raw_samples:
            self.samples.append((raw_sample, int(raw_sample.split('\\')[-3])))

    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        if self.mode == 'train':
            random.shuffle(self.samples)
            
        image_dir, label = self.samples[index]
        images = glob.glob(image_dir)

        
        transformations = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(degrees=45),
            transforms.ToTensor()            
        ])
        
        array = []
        
        for i, image_path in enumerate(images):
            image = Image.open(image_path)
            image = np.array(image)
            r, c, _ = image.shape
            for i in range(0,32*24,32):
                for j in range(0,32*28,32):
                    array.append(transformations(image[i:i+32, j:j+32, :]))

                    
                    
        array = tuple(array)

        array = torch.stack(array, 0)
        
        return (array, label)

In [ ]:
''' Attention model definition along with loss function and function to compute classification error '''
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 500
        self.D = 128
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(20, 50, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(50 * 4 * 4, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.squeeze(0)
        H = self.feature_extractor_part1(x)
        H = H.view(-1, 50 * 4 * 4)
        H = self.feature_extractor_part2(H)  

        A = self.attention(H)  
        A = torch.transpose(A, 1, 0)  
        A = F.softmax(A, dim=1)  

        M = torch.mm(A, H)  

        Y_prob = self.classifier(M)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat, A
    
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat, _ = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _, A = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))  

        return neg_log_likelihood, A

In [6]:
''' Running the training and testing of model to get the accuracy of model '''
import numpy as np
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import os
import glob
from PIL import Image
import numpy as np
from torchvision import datasets, transforms
from torchvision import models
import random


import argparse

parser = argparse.ArgumentParser(description='USCB-Augmented-classification')

parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')

args, unknown = parser.parse_known_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()




print(' epoch_{} learning_rate_{}'.format( 100, 0.0001))


torch.manual_seed(1)
if args.cuda:
    torch.cuda.manual_seed(1)
    print('\nGPU is ON!')

print('Load Train and Test Set')
loader_kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}


data_path_test = 'USCB/test/*/*/*/*.tif'
original_train_data = PatchMethod()
aug_train_data = AugPatchMethod()
data=torch.utils.data.ConcatDataset([original_train_data,aug_train_data])
original_val_data =PatchMethod(root = data_path_test, mode = 'test')
aug_val_data = AugPatchMethod(root = data_path_test, mode = 'test')
val_data=torch.utils.data.ConcatDataset([original_val_data,aug_val_data])


train_loader = torch.utils.data.DataLoader(data, shuffle = True, num_workers = 0, batch_size = 1)
test_loader = torch.utils.data.DataLoader(val_data, shuffle = False, num_workers = 0, batch_size = 1)


print('Init Model')
model = Attention()
if args.cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), weight_decay=0.0005)


def train(epoch):
    model.train()
    train_loss = 0.
    train_error = 0.
    correct_label_pred = 0
    for batch_idx, (data, label) in enumerate(train_loader):
 
        bag_label = label[0]
        if args.cuda:
            data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        data = data.squeeze(0)

        optimizer.zero_grad()
        loss, _ = model.calculate_objective(data, bag_label)
        train_loss += loss.data[0]
        error, predicted_label_train = model.calculate_classification_error(data, bag_label)
        correct_label_pred += (int(bag_label) == int(predicted_label_train))
        train_error += error
        loss.backward()
        optimizer.step()

    
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    train_acc = (1 - train_error)*100

 

    result_train = 'Epoch: {}, Loss: {:.4f}, Train error: {:.4f}, Train accuracy: {:.2f}'.format(epoch, train_loss.cpu().numpy()[0], train_error, train_acc)

    print(result_train)
    return result_train

def test(epoch):
    model.eval()
    test_loss = 0.
    test_error = 0.
    for batch_idx, (data, label) in enumerate(test_loader):
        

        bag_label = label[0]
        instance_labels = label[0]
        if args.cuda:
            data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss, attention_weights = model.calculate_objective(data, bag_label)
        test_loss += loss.data[0]
        error, predicted_label = model.calculate_classification_error(data, bag_label)
        test_error += error

        
        if batch_idx < 1: 
            bag_level = (bag_label.cpu().data.numpy(), int(predicted_label.cpu().data.numpy()))
      

    test_error /= len(test_loader)
    test_loss /= len(test_loader)
    test_acc = (1 - test_error)*100    

   
    result_test = 'Epoch: {}, Loss: {:.4f}, test error: {:.4f}, test accuracy: {:.2f}'.format(epoch, test_loss.cpu().numpy()[0], test_error, test_acc)
    print(result_test)
    return result_test



if __name__ == "__main__":
    
    for epoch in range(1, 100 + 1):
        train_result = train(epoch)
        test_result = test(epoch)
    



 epoch_100 learning_rate_0.0001

GPU is ON!
Load Train and Test Set
Init Model
Epoch: 1, Loss: 0.6941, Train error: 0.4875, Train accuracy: 51.25
Epoch: 1, Loss: 0.6925, test error: 0.5000, test accuracy: 50.00
Epoch: 2, Loss: 0.6800, Train error: 0.4000, Train accuracy: 60.00
Epoch: 2, Loss: 0.6982, test error: 0.5000, test accuracy: 50.00
Epoch: 3, Loss: 0.6870, Train error: 0.4125, Train accuracy: 58.75
Epoch: 3, Loss: 0.6940, test error: 0.5000, test accuracy: 50.00
Epoch: 4, Loss: 0.6855, Train error: 0.4250, Train accuracy: 57.50
Epoch: 4, Loss: 0.6921, test error: 0.5000, test accuracy: 50.00
Epoch: 5, Loss: 0.6933, Train error: 0.4875, Train accuracy: 51.25
Epoch: 5, Loss: 0.6888, test error: 0.5000, test accuracy: 50.00
Epoch: 6, Loss: 0.6750, Train error: 0.4250, Train accuracy: 57.50
Epoch: 6, Loss: 0.6890, test error: 0.5000, test accuracy: 50.00
Epoch: 7, Loss: 0.7066, Train error: 0.6000, Train accuracy: 40.00
Epoch: 7, Loss: 0.6924, test error: 0.5000, test accuracy: 50.

Epoch: 62, Loss: 0.5528, Train error: 0.3000, Train accuracy: 70.00
Epoch: 62, Loss: 0.4191, test error: 0.1750, test accuracy: 82.50
Epoch: 63, Loss: 0.4397, Train error: 0.2250, Train accuracy: 77.50
Epoch: 63, Loss: 0.4682, test error: 0.2250, test accuracy: 77.50
Epoch: 64, Loss: 0.3698, Train error: 0.1500, Train accuracy: 85.00
Epoch: 64, Loss: 0.4765, test error: 0.2375, test accuracy: 76.25
Epoch: 65, Loss: 0.4524, Train error: 0.2000, Train accuracy: 80.00
Epoch: 65, Loss: 0.4327, test error: 0.2000, test accuracy: 80.00
Epoch: 66, Loss: 0.4542, Train error: 0.2000, Train accuracy: 80.00
Epoch: 66, Loss: 0.4166, test error: 0.1625, test accuracy: 83.75
Epoch: 67, Loss: 0.3625, Train error: 0.1500, Train accuracy: 85.00
Epoch: 67, Loss: 0.3988, test error: 0.1875, test accuracy: 81.25
Epoch: 68, Loss: 0.4107, Train error: 0.1875, Train accuracy: 81.25
Epoch: 68, Loss: 0.4112, test error: 0.1875, test accuracy: 81.25
Epoch: 69, Loss: 0.4558, Train error: 0.2125, Train accuracy: 